## Notebook de Analisis de Afluenta

In [1]:
import pandas as pd
import numpy as np


In [2]:
!ls

afluenta.csv		    LenderAccountBug.ipynb	README.md
afluenta - Sheet1.csv	    LenderAccountMovements.xls
fastInterestAnalysis.ipynb  LenderProfitProjection.xls


In [4]:
pd.read_excel?

In [12]:
df = pd.read_csv('LenderAccountMovements.csv',index_col=None,sep="|")
df.columns

Index(['Fecha', 'Operación', 'Débitos', 'Créditos', 'Saldo'], dtype='object')

In [13]:
df.head()

,Fecha,Operación,Débitos,Créditos,Saldo
0,22/11/2016 23:24,"Comisión Afluenta: #AR-221607-G0009 (4/24) $ 6,90",0.14,NaN,21928.17
1,22/11/2016 23:24,"Retorno por cuota: #AR-221607-G0009 (4/24) $ 6,90",NaN,4.14,21928.31
2,22/11/2016 23:21,"Comisión Afluenta: #AR-301609-G0003 (2/48) $ 8,79",0.18,NaN,21924.17
3,22/11/2016 23:21,"Retorno por cuota: #AR-301609-G0003 (2/48) $ 8,79",NaN,7.11,21924.35
4,21/11/2016 10:51,Comisión Afluenta: #AR-241607-I0004 (3/24) $ 1...,0.26,NaN,21917.24


In [17]:
# get datatypes and null vals
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 5 columns):
Fecha        796 non-null object
Operación    796 non-null object
Débitos      400 non-null float64
Créditos     396 non-null float64
Saldo        796 non-null float64
dtypes: float64(3), object(2)
memory usage: 31.2+ KB


In [30]:
df.columns

Index(['Fecha', 'Operación', 'Débitos', 'Créditos', 'Saldo'], dtype='object')

In [18]:
## fill null vals with 0
numeric_cols = df.columns[-3:]
for col in numeric_cols:
    df[col] = df[col].fillna(0)

#set datetime col
col = 'Fecha'
df[col] = pd.to_datetime(df[col])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 5 columns):
Fecha        796 non-null datetime64[ns]
Operación    796 non-null object
Débitos      796 non-null float64
Créditos     796 non-null float64
Saldo        796 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 31.2+ KB


In [35]:
df = df.set_index('Fecha')

# check accounting i.e. inputs - outputs == current_holdings

In [36]:
input_money = df['Créditos'].sum() 
output_money = df['Débitos'].sum()  
last_balance = df['Saldo'].iloc[0]

In [51]:
last_balance, input_money, output_money

(21909.759999999998, 22071.050000000003, 384.01999999999998)

In [40]:
input_money - output_money == last_balance

True

## check same accounting until certain date limit

In [49]:
def net_accounting(df, filter_date=None):
    if filter_date:
        df = df[df.index<= filter_date]     
    input_money = df['Créditos'].sum() 
    output_money = df['Débitos'].sum()  
    last_balance = df['Saldo'].iloc[0]

    return (input_money, output_money, last_balance)

In [96]:
input_money, output_money, last_balance = net_accounting(df, filter_date=pd.datetime(2016,11,15))

In [97]:
last_balance, input_money, output_money

(21808.07, 21929.82, 344.48000000000002)

In [98]:
input_money - output_money == last_balance

False

# Bug found ??

In [99]:
df.head(15)

,Operación,Débitos,Créditos,Saldo
Fecha,,,,
2016-11-22 23:24:00,"Comisión Afluenta: #AR-221607-G0009 (4/24) $ 6,90",0.14,0.00,21928.17
2016-11-22 23:24:00,"Retorno por cuota: #AR-221607-G0009 (4/24) $ 6,90",0.00,4.14,21928.31
2016-11-22 23:21:00,"Comisión Afluenta: #AR-301609-G0003 (2/48) $ 8,79",0.18,0.00,21924.17
2016-11-22 23:21:00,"Retorno por cuota: #AR-301609-G0003 (2/48) $ 8,79",0.00,7.11,21924.35
2016-11-21 10:51:00,Comisión Afluenta: #AR-241607-I0004 (3/24) $ 1...,0.26,0.00,21917.24
2016-11-21 10:51:00,Retorno por cuota: #AR-241607-I0004 (3/24) $ 1...,0.00,7.74,21917.50
2016-11-18 14:45:00,Comisión Afluenta: #AR-211601-I0006 (9/24) $ 1...,0.27,0.00,21909.76
2016-11-18 14:45:00,Retorno por cuota: #AR-211601-I0006 (9/24) $ 1...,0.00,6.69,21910.03
2016-11-18 14:43:00,Comisión Afluenta: #AR-191607-G0009 (3/48) $ 2...,0.42,0.00,21903.34


## "Rolling" check for every day in the dataset

In [104]:
start_date = df.index.values.min()
end_date = df.index.values.max()

In [105]:
#cast to datetime type
import datetime
start_date = datetime.datetime.utcfromtimestamp(start_date.astype('O')/1e9)
end_date = datetime.datetime.utcfromtimestamp(end_date.astype('O')/1e9)

In [106]:
from datetime import timedelta
delta = timedelta(days = 1)

current_date = start_date
results = []
while current_date < end_date:
    current_date = current_date +  delta
    input_money, output_money, last_balance = net_accounting(df, filter_date=pd.datetime(2016,11,21))
    results+= [input_money - output_money == last_balance]
    

In [107]:
results

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
